The total number of images, minumum, maximum and average no. of images per cattle.  
  
For dataset 1

In [1]:
import os
from collections import Counter

dataset_path = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\1_base_256"

# Count cattle and images
cattle_folders = os.listdir(dataset_path)
image_counts = []

for cattle_id in cattle_folders:
    cattle_path = os.path.join(dataset_path, cattle_id)
    if os.path.isdir(cattle_path):
        images = [f for f in os.listdir(cattle_path) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        image_counts.append(len(images))

print(f"Total cattle: {len(cattle_folders)}")
print(f"Total images: {sum(image_counts)}")
print(f"Images per cattle: Min={min(image_counts)}, Max={max(image_counts)}, Avg={sum(image_counts)/len(image_counts):.1f}")

Total cattle: 455
Total images: 2747
Images per cattle: Min=1, Max=27, Avg=6.0


For dataset 2

In [2]:
import os
from collections import Counter

dataset_path = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\Second dataset dataset\2_base_256"

# Count cattle and images
cattle_folders = os.listdir(dataset_path)
image_counts = []

for cattle_id in cattle_folders:
    cattle_path = os.path.join(dataset_path, cattle_id)
    if os.path.isdir(cattle_path):
        images = [f for f in os.listdir(cattle_path) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        image_counts.append(len(images))

print(f"Total cattle: {len(cattle_folders)}")
print(f"Total images: {sum(image_counts)}")
print(f"Images per cattle: Min={min(image_counts)}, Max={max(image_counts)}, Avg={sum(image_counts)/len(image_counts):.1f}")

Total cattle: 268
Total images: 4923
Images per cattle: Min=4, Max=70, Avg=18.4


Number of the class with less than 5 images.
For dataset 1


In [8]:
import os
from collections import Counter

# Path to your dataset (change this)
dataset_path = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\1_base_256"

# Dictionary to store counts
class_counts = {}

# Loop over all class folders
for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_folder):
        num_images = len([f for f in os.listdir(class_folder) 
                          if os.path.isfile(os.path.join(class_folder, f))])
        class_counts[class_name] = num_images

# Count how many classes fall into 1,2,3,4,5 images
bucket_counts = Counter(class_counts.values())

print("📌 Number of classes with 1–5 images:")
for i in range(1, 6):
    print(f"{i} image(s): {bucket_counts.get(i, 0)} classes")


📌 Number of classes with 1–5 images:
1 image(s): 55 classes
2 image(s): 39 classes
3 image(s): 85 classes
4 image(s): 33 classes
5 image(s): 25 classes


For dataset 2

In [7]:
import os
from collections import Counter

# Path to your dataset (change this)
dataset_path = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\Second dataset dataset\2_base_256"

# Dictionary to store counts
class_counts = {}

# Loop over all class folders
for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_folder):
        num_images = len([f for f in os.listdir(class_folder) 
                          if os.path.isfile(os.path.join(class_folder, f))])
        class_counts[class_name] = num_images

# Count how many classes fall into 1,2,3,4,5 images
bucket_counts = Counter(class_counts.values())

print("📌 Number of classes with 1–5 images:")
for i in range(1, 6):
    print(f"{i} image(s): {bucket_counts.get(i, 0)} classes")


📌 Number of classes with 1–5 images:
1 image(s): 0 classes
2 image(s): 0 classes
3 image(s): 0 classes
4 image(s): 8 classes
5 image(s): 4 classes


-----------------------------------------------------------------------------------------------------------------
## For dataset 1
Since i have the dataset already in the grayscale so for the preprocessing the only thing i have to do is augumentation for the class.  
As the test and validate folder should have the original image of the images not the augumentation.So, according to the dataset it is divided into the train,test and validate in the following way:  

-  Decide test, val, train_real counts per class:

    - If n >= 10:  
        test = max(1, round(0.15 * n))  
        val = max(1, round(0.15 * n))  
        train_real = n - test - val  

    - If 4 <= n < 10:
        test = 1, val = 1, train_real = n - 2

    - If n == 3:
        test = 1, val = 1, train_real = 1

    - If n == 2:
        test = 1, val = 0, train_real = 1 (we will create 1 augmented image for val later)

    - If n == 1:
        test = 1, val = 0, train_real = 0 (train images will be created by offline augmentation)



In [ ]:
#!/usr/bin/env python3
"""
Script A — split originals into train/val/test per-class using robust rules.

Usage: edit DATA_ROOT and OUT_ROOT and run.
"""
import os
import shutil
import random
from pathlib import Path
import csv

random.seed(42)

# -------- USER SETTINGS --------
DATA_ROOT = Path(r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\1_base_256")    # folder with class subfolders
OUT_ROOT  = Path(r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_1")  # will contain train/val/test subfolders
TARGET_TRAIN = 10   # target training images per class after offline augmentation
MIN_VAL_AUG = True  # if val missing, use small augmented image later (flagging)
# -------------------------------

def decide_counts(n):
    if n >= 10:
        test = max(1, round(0.15 * n))
        val  = max(1, round(0.15 * n))
        train = n - test - val
    elif 4 <= n < 10:
        test = 1; val = 1; train = n - 2
    elif n == 3:
        test = 1; val = 1; train = 1
    elif n == 2:
        test = 1; val = 0; train = 1
    elif n == 1:
        test = 1; val = 0; train = 0
    else:
        test=0; val=0; train=0
    return int(max(test,0)), int(max(val,0)), int(max(train,0))

def ensure_dirs(p):
    (p / "train").mkdir(parents=True, exist_ok=True)
    (p / "val").mkdir(parents=True, exist_ok=True)
    (p / "test").mkdir(parents=True, exist_ok=True)

def main():
    ensure_dirs(OUT_ROOT)
    rows = []
    for cls_dir in sorted(DATA_ROOT.iterdir()):
        if not cls_dir.is_dir(): continue
        imgs = sorted([p for p in cls_dir.iterdir() if p.suffix.lower() in (".jpg",".jpeg",".png",".bmp",".tif",".tiff")])
        n = len(imgs)
        test_c, val_c, train_c = decide_counts(n)
        # shuffle then select
        random.shuffle(imgs)
        test_imgs = imgs[:test_c]
        val_imgs = imgs[test_c:test_c+val_c]
        train_imgs = imgs[test_c+val_c:test_c+val_c+train_c]
        # fallback: if val_c==0 but MIN_VAL_AUG True, we will mark val as augmented later
        # Copy files
        for p in train_imgs:
            out = OUT_ROOT / "train" / cls_dir.name
            out.mkdir(parents=True, exist_ok=True)
            shutil.copy2(p, out / p.name)
        for p in val_imgs:
            out = OUT_ROOT / "val" / cls_dir.name
            out.mkdir(parents=True, exist_ok=True)
            shutil.copy2(p, out / p.name)
        for p in test_imgs:
            out = OUT_ROOT / "test" / cls_dir.name
            out.mkdir(parents=True, exist_ok=True)
            shutil.copy2(p, out / p.name)

        to_generate = max(0, TARGET_TRAIN - max(0, train_c))
        rows.append({
            "class": cls_dir.name,
            "n_original": n,
            "train_real": len(train_imgs),
            "val_real": len(val_imgs),
            "test_real": len(test_imgs),
            "to_generate_train": to_generate,
            "val_is_augmented_needed": (len(val_imgs)==0 and MIN_VAL_AUG)
        })

    # Save CSV summary
    csv_path = OUT_ROOT / "split_summary.csv"
    with open(csv_path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()) if rows else [])
        writer.writeheader()
        for r in rows:
            writer.writerow(r)

    print("Split done. Summary saved to:", csv_path)

if __name__ == "__main__":
    main()


Split done. Summary saved to: C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\split_summary.csv


After splitting, it compute the no. of images needed in the train folder to be 10 and then generate the images by the augumenatition technique. Check the split_summary.csv if it is true for any class then it apply the mild augumentation to generate the validation image. 


In [ ]:
#!/usr/bin/env python3
"""
Script B — offline augment to reach target_train per class.

Usage: edit PROCESSED_ROOT and adjust parameters.
"""
import cv2
import numpy as np
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
import csv
import random

random.seed(42)

# -------- USER SETTINGS --------
PROCESSED_ROOT = Path(r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_1")  # output of Script A
SUMMARY_CSV = PROCESSED_ROOT / "split_summary.csv"
TARGET_TRAIN = 10
AUG_SAVE_QUALITY = 95
# -------------------------------

# augmentation pipeline for offline generation (diverse)
aug_pipeline = A.Compose([
    A.RandomResizedCrop(size=(224, 224), scale=(0.8,1.0), ratio=(0.9,1.1)),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.08, rotate_limit=12, p=0.6),
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
    A.GaussNoise(var_limit=(5.0, 30.0), p=0.3),
    A.OneOf([A.MotionBlur(blur_limit=3), A.MedianBlur(blur_limit=3), A.GaussianBlur(blur_limit=3)], p=0.2),
    A.RandomGamma(p=0.3),
    A.CoarseDropout(max_holes=1, max_height=32, max_width=32, p=0.25),
])

# mild augmentation for validation (if needed) -> minimal transformation
val_aug = A.Compose([
    A.RandomResizedCrop(size=(224, 224), scale=(0.95,1.0), ratio=(0.98,1.02)),
    A.HorizontalFlip(p=0.5)
])

def read_summary():
    rows = []
    with open(SUMMARY_CSV, newline="") as f:
        r = csv.DictReader(f)
        for row in r:
            rows.append(row)
    return rows

def list_train_real(cls):
    p = PROCESSED_ROOT / "train" / cls
    if not p.exists(): return []
    return sorted([x for x in p.iterdir() if x.suffix.lower() in (".jpg",".jpeg",".png",".bmp",".tif",".tiff")])

def create_augmented_image(img_path, pipeline):
    img = cv2.imread(str(img_path))
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out = pipeline(image=img)
    aug = out['image']
    aug_bgr = cv2.cvtColor(aug, cv2.COLOR_RGB2BGR)
    return aug_bgr

def save_aug(img_bgr, out_path):
    cv2.imwrite(str(out_path), img_bgr, [int(cv2.IMWRITE_JPEG_QUALITY), AUG_SAVE_QUALITY])

def main():
    rows = read_summary()
    for r in rows:
        cls = r["class"]
        to_gen = int(r["to_generate_train"])
        val_aug_needed = r.get("val_is_augmented_needed","False").lower() in ("true","1","yes")
        train_dir = PROCESSED_ROOT / "train" / cls
        train_dir.mkdir(parents=True, exist_ok=True)
        existing = list_train_real(cls)
        existing_count = len(existing)
        # If no real training images exist, use test/val images as source for augmentation generation (do not move them)
        source_pool = existing.copy()
        if len(source_pool) == 0:
            # try using val or test if present (but DO NOT move them, use only as augmentation source)
            for alt in ("val", "test"):
                altdir = PROCESSED_ROOT / alt / cls
                if altdir.exists():
                    source_pool += sorted([x for x in altdir.iterdir() if x.suffix.lower() in (".jpg",".jpeg",".png")])
            if len(source_pool) == 0:
                print(f"[WARN] No source images found for class {cls}; skipping augmentation.")
                continue

        # Generate images
        i = 0
        while to_gen > 0:
            src = random.choice(source_pool)
            aug_img = create_augmented_image(src, aug_pipeline)
            if aug_img is None:
                print(f"[WARN] failed to read {src}")
                break
            out_name = f"aug_train_{i:04d}__{src.stem}.jpg"
            save_aug(aug_img, train_dir / out_name)
            i += 1
            to_gen -= 1

        # If val augmentation needed (mild), create a single augmented val image and save to val folder
        if val_aug_needed:
            val_dir = PROCESSED_ROOT / "val" / cls
            val_dir.mkdir(parents=True, exist_ok=True)
            # pick a source from test (prefer) or train
            srcs = sorted([x for x in (PROCESSED_ROOT / "test" / cls).glob("*")]) or sorted([x for x in (PROCESSED_ROOT / "train" / cls).glob("*")])
            if srcs:
                src = random.choice(srcs)
                aug_img = create_augmented_image(src, val_aug)
                if aug_img is not None:
                    save_aug(aug_img, val_dir / f"aug_val__{src.stem}.jpg")
            else:
                print(f"[WARN] no source to create val augmentation for {cls}")

    print("Offline augmentation complete.")

if __name__ == "__main__":
    main()


C:\Users\Amrit Shah\AppData\Local\Temp\ipykernel_15292\3649991722.py:30: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 30.0), p=0.3),
C:\Users\Amrit Shah\AppData\Local\Temp\ipykernel_15292\3649991722.py:33: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=1, max_height=32, max_width=32, p=0.25),


Offline augmentation complete.


## For dataset 2
As the dataset has minimum 4 images per class so here i have splitted the dataset in the following way:  

For classes with ≥7 images: split 70/15/15 normally (round counts).  

For classes with 4–7 images:  
    - At least 1 image in the validate and test

In [21]:
import os
import shutil
import random

# -------------------------
# CONFIG
# -------------------------
SOURCE_DIR = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\Second dataset dataset\2_base_256"
TARGET_DIR = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_2"
MIN_TRAIN_IMAGES = 10   # minimum images per class in train after augmentation

# -------------------------
# STEP 1: SPLIT DATASET SMARTLY
# -------------------------
def smart_split(source_dir, target_dir):
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(target_dir, split), exist_ok=True)

    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        images = [f for f in os.listdir(class_path) if f.lower().endswith((".png", ".jpg", ".jpeg"))]
        if len(images) == 0:
            continue

        random.shuffle(images)
        n = len(images)

        # Small classes
        if n < 4:
            train_imgs = images
            val_imgs = []
            test_imgs = []
        elif 4 <= n < 7:
            train_count = max(2, n - 2)
            val_count = 1
            test_count = n - train_count - val_count
            train_imgs = images[:train_count]
            val_imgs = images[train_count:train_count + val_count]
            test_imgs = images[train_count + val_count:]
        else:  # n >= 10
            train_count = int(n * 0.7)
            val_count = int(n * 0.15)
            test_count = n - train_count - val_count
            train_imgs = images[:train_count]
            val_imgs = images[train_count:train_count + val_count]
            test_imgs = images[train_count + val_count:]

        # Copy images
        for split_name, split_imgs in zip(["train", "val", "test"], [train_imgs, val_imgs, test_imgs]):
            split_class_dir = os.path.join(target_dir, split_name, class_name)
            os.makedirs(split_class_dir, exist_ok=True)
            for img in split_imgs:
                shutil.copy(os.path.join(class_path, img), os.path.join(split_class_dir, img))

    print("✅ Smart split done!")

if __name__ == "__main__":
    smart_split(SOURCE_DIR, TARGET_DIR)


✅ Smart split done!


After splitting, it compute the no. of images required in the train folder to be 10 and then generate the remaining images by the augumenatition technique.

In [4]:
import os
import shutil
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
TARGET_DIR = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_2"
MIN_TRAIN_IMAGES = 10 

#Augmentation settings
datagen = ImageDataGenerator( 
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    zoom_range=0.1, 
    horizontal_flip=True, 
    fill_mode='nearest' )

def augment_train(target_dir, min_train_images=MIN_TRAIN_IMAGES):
    train_dir = os.path.join(target_dir, "train")
    for class_name in os.listdir(train_dir):
        class_path = os.path.join(train_dir, class_name)
        images = [f for f in os.listdir(class_path) if f.lower().endswith((".png", ".jpg", ".jpeg"))]
        n = len(images)

        if n >= min_train_images:
            continue

        needed = min_train_images - n
        print(f"Augmenting {class_name}: {n} → {min_train_images} images")

        i = 0
        while needed > 0:
            img_path = os.path.join(class_path, images[i % n])
            img = load_img(img_path)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            # generate 1 augmented image
            for batch in datagen.flow(x, batch_size=1, save_to_dir=class_path,
                                      save_prefix='aug', save_format='png'):
                needed -= 1
                i += 1
                break  # only generate one per loop

    print("✅ Augmentation done!")

# -------------------------
# MAIN
# -------------------------
if __name__ == "__main__":
    augment_train(TARGET_DIR)

Augmenting cattle_0100: 5 → 10 images
Augmenting cattle_0200: 7 → 10 images
Augmenting cattle_0400: 4 → 10 images
Augmenting cattle_0500: 9 → 10 images
Augmenting cattle_0900: 8 → 10 images
Augmenting cattle_1000: 8 → 10 images
Augmenting cattle_1100: 7 → 10 images
Augmenting cattle_1200: 7 → 10 images
Augmenting cattle_1300: 8 → 10 images
Augmenting cattle_1400: 9 → 10 images
Augmenting cattle_1500: 4 → 10 images
Augmenting cattle_1600: 9 → 10 images
Augmenting cattle_1700: 8 → 10 images
Augmenting cattle_1900: 5 → 10 images
Augmenting cattle_2000: 9 → 10 images
Augmenting cattle_2100: 2 → 10 images
Augmenting cattle_2200: 4 → 10 images
Augmenting cattle_2220: 4 → 10 images
Augmenting cattle_2320: 9 → 10 images
Augmenting cattle_2510: 7 → 10 images
Augmenting cattle_2740: 5 → 10 images
Augmenting cattle_2930: 4 → 10 images
Augmenting cattle_3100: 9 → 10 images
Augmenting cattle_3300: 9 → 10 images
Augmenting cattle_3400: 4 → 10 images
Augmenting cattle_3420: 2 → 10 images
Augmenting c

Detail of the dataset 2 in the csv file.

In [6]:
import os
import csv

# -------------------------
# CONFIG
# -------------------------
TARGET_DIR = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_2"
CSV_PATH   = os.path.join(TARGET_DIR, "dataset_split.csv")

def generate_csv_from_split(target_dir, csv_path):
    rows = [("split", "class", "filename")]  # header row
    
    for split in ["train", "val", "test"]:
        split_dir = os.path.join(target_dir, split)
        if not os.path.exists(split_dir):
            continue

        for class_name in os.listdir(split_dir):
            class_path = os.path.join(split_dir, class_name)
            if not os.path.isdir(class_path):
                continue

            for img in os.listdir(class_path):
                if img.lower().endswith((".png", ".jpg", ".jpeg")):
                    rows.append((split, class_name, img))

    # save CSV
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    print(f"✅ CSV generated at {csv_path}")

if __name__ == "__main__":
    generate_csv_from_split(TARGET_DIR, CSV_PATH)


✅ CSV generated at C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_2\dataset_split.csv
